<a href="https://colab.research.google.com/github/takatakamanbou/AdvML/blob/2024/AdvML2024_ex10notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AdvML ex10notebookA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/AdvML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?AdvML)




板書や口頭で補足する前提なので，この notebook だけでは説明が不完全です．


---
## 統計的識別入門 (2)
---


---
### 準備


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture

---
### 生成モデルを用いた識別の例1: 2次元データ

2次元のデータを生成モデルを用いて識別する実験をやってみよう．

まずは学習データの準備．

In [ ]:
# データの読み込み
K = 3 # クラス数
D = 2 # 特徴次元数
df = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/2dim3class.csv')

X = df.drop(columns='label').to_numpy()
y = df['label'].to_numpy()

df

In [ ]:
# グラフを描く
fig, ax = plt.subplots()
for k in range(K):
    ax.scatter(X[y==k, 0], X[y==k, 1], label=f'class{k}')
xmin, xmax = -5, 5
ymin, ymax = -5, 5
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal')
ax.legend()
plt.show()

事前確率は，学習データの出現頻度で推定することにする．

In [ ]:
# 事前確率を推定
py = np.empty(K)
for k in range(K):
    py[k] = np.sum(y==k) / len(y)
print('### 事前確率')
print('p(y) = ', py)


クラスごとの分布には2次元正規分布を当てはめる．

In [ ]:
# p(x|y) を正規分布でモデル化（平均，共分散行列の推定）
mu = np.empty((K, D))
cov = np.empty((K, D, D))
for k in range(K):
    XX = X[y==k, :]
    mu[k] = np.mean(XX, axis=0)
    cov[k] = (XX - mu[k]).T @ (XX - mu[k]) / XX.shape[0]
    print(f'### class {k} の正規分布のパラメータ')
    print(mu[k])
    print(cov[k])
    print()

次のコードを実行すると，得られたモデルを用いて，2次元平面上の各点での事後確率を求め，それを可視化することができる．

In [ ]:
# グラフ描画用のグリッドデータの作成
x_mesh, y_mesh = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
X_mesh = np.dstack((x_mesh, y_mesh))

# 事後確率の推定
p = np.empty((K, X_mesh.shape[0]*X_mesh.shape[1]))
for k in range(K):
    p[k, :] = py[k] * multivariate_normal.pdf(X_mesh, mean=mu[k], cov=cov[k]).reshape(-1)
p /= np.sum(p, axis=0)
pp = p.reshape((K, X_mesh.shape[0], X_mesh.shape[1]))

# グラフ
fig = plt.figure(figsize=(9, 6))

# Gaussian を当てはめた結果
ax0 = fig.add_subplot(121)
for k in range(K):
    ax0.scatter(X[y==k, 0], X[y==k, 1])
    ax0.contour(x_mesh, y_mesh, multivariate_normal.pdf(X_mesh, mean=mu[k], cov=cov[k]))
ax0.set_xlim(xmin, xmax)
ax0.set_ylim(ymin, ymax)
ax0.set_aspect('equal')

# 事後確率の可視化
cmap = ['Blues', 'Oranges', 'Greens']
ax1 = fig.add_subplot(122)
for k in range(K):
    ax1.scatter(X[y==k, 0], X[y==k, 1])
    ax1.contourf(x_mesh, y_mesh, pp[k], levels=[0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.set_aspect('equal')

plt.show()

---
### 多次元正規分布についての補足

$$
p(\mathbf{x}; \mathbf{\mu}, \Sigma) = \frac{1}{\sqrt{(2\pi)^D|\Sigma|}} \exp{ \left( -\frac{1}{2} (\mathbf{x}-\mathbf{\mu})^{\top}\Sigma^{-1}(\mathbf{x}-\mathbf{\mu}) \right) }
$$


---
### 生成モデルを用いた識別の例2: 手書き数字識別

手書き数字のデータを生成モデルを用いて識別する実験をやってみよう．

まずは学習データの準備．

In [ ]:
# 手書き数字データの入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/minimnist.npz
rv = np.load('minimnist.npz')
datL = rv['datL'].astype(float)
labL = rv['labL']
datT = rv['datT'].astype(float)
labT = rv['labT']
print(datL.shape, labL.shape, datT.shape, labT.shape)

K = 10 # クラス数

# 学習データの用意
NL, D = datL.shape # 学習データの数と次元数
XL = datL/255
yL = labL

# テストデータの用意
NT, _ = datT.shape # テストデータの数
XT = datT/255
yT = labT

学習データは 5000 個．データの次元数は 784．

In [ ]:
# 事前確率を推定
py = np.empty(K)
for k in range(K):
    py[k] = np.sum(yL==k) / NL
print('### 事前確率')
print('p(y) = ', py)

2次元の例では，クラスごとに正規分布を当てはめる際に，平均と分散共分散行列を定義通り求めていたが，ここでは [sklearn.mixture.GaussianMixture](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html) を用いることにする．
そうする理由は後述する．

GaussianMixture は，本来は混合正規分布（次回解説予定）のためのクラスであるが，`n_components=1` とすることで， 1つの正規分布を当てはめることができる．
`covariance_type` や `reg_covar` というオプションの意味は後述する．

In [ ]:
# p(x|y) を正規分布でモデル化（平均，共分散行列の推定）
mu = np.empty((K, D))
Gaussian = np.empty(K, dtype=object)
for k in range(K):
    XX = XL[labL==k, :]
    Gaussian[k] = GaussianMixture(n_components=1, covariance_type='full', reg_covar=1e-06)
    Gaussian[k].fit(XX)
    mu[k] = Gaussian[k].means_[0]

次のコードセルを実行すると，クラスごとの正規分布の平均を画像として可視化できる．

In [ ]:
# クラスごとの正規分布の平均を可視化
fig, ax = plt.subplots(1, K, figsize=(8, 2))
for k in range(K):
    img = mu[k].reshape((28, 28))
    ax[k].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
    ax[k].axis('off')
    ax[k].set_title(k)

fig.tight_layout()
plt.show()

学習データ，テストデータのそれぞれを識別させてみよう．

In [ ]:
### 学習データの識別

# 対数尤度を算出
LLL = np.empty([NL, K])
for k in range(K):
    # log p(y|x) = log p(y) + log p(x|y)
    LLL[:, k] = np.log(py[k]) + Gaussian[k].score_samples(XL)

# 尤度最大のクラスに識別
y_predict = np.argmax(LLL, axis=1)
ncL = np.sum(yL == y_predict)

### テストデータの識別

# 対数尤度を算出
LLT = np.empty([NT, K])
for k in range(K):
    # log p(y|x) = log p(y) + log p(x|y)
    LLT[:, k] = np.log(py[k]) + Gaussian[k].score_samples(XT)

# 尤度最大のクラスに識別
y_predict = np.argmax(LLT, axis=1)
ncT = np.sum(yT == y_predict)

print(f'学習: {ncL}/{NL} = {ncL/NL}')
print(f'テスト: {ncT}/{NT} = {ncT/NT}')

この問題の場合に分散共分散行列を定義通り計算するとうまくいかない理由は...

分散共分散行列の正則化と `reg_covar`

分散共分散行列の形を制約する話と `covariance_type`



---
### 判別分析や最短距離法との関係